<a href="https://colab.research.google.com/github/KodumuruRaja/Decision-Tree-Algorithms/blob/main/Decision-Tree-program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
training_data=[['Green',3,'Mango'],['Yellow',3,'Mango'],['Red',1,'Grape'],['Red',1,'Grape'],['Yellow',3,'Lemon']]

In [ ]:
# column labels
header=['color','diameter','label']

In [ ]:
# finding unique values for a column in a data
def unique(row,col):
  return set([row[col] for row in rows])

In [ ]:
# it counts the each type of example in data
def class_counts(rows):
  counts={} # dictionary of label count
  for row in rows: # in our dataset format, the label is always the last column
    label=row[-1]
    if label not in counts:
      counts[label]=0
    counts[label]+=1
  return counts

In [ ]:
# To test if a value is numeric
def is_numeric(value):
  return isinstance(value,int) or isinstance(value,float)

In [ ]:
# Quest is used for partitioning of data
class Question:
  def __init__(self,column,value):
    self.column=column
    self.value=value
# match method is used to compare the feature value in an example to the feature value stored in the question
  def match(self,example):
    val=example[self.column]
    if is_numeric(val):
      return val>=self.value
    else:
      return val==self.value
# It is a helper method to print the question in a readable format
    def __repr__(self):
      condition="=="
      if is_numeric(self.value):
        condition=">="
      return "Is %s %s %s?" % (header[self.column],condition,str(self.value))


In [ ]:
# It checks each row in the data if it matches the question. If so,add it to 'true_rows',otherwise,add it to 'false_rows'
def partition(rows,question):
  true_rows,false_rows=[],[]
  for row in rows:
    if question.match(row):
      true_rows.append(row)
    else:
      false_rows.append(row)
  return true_rows,false_rows

In [ ]:
# calculate gini impurity for list of rows
def gini(rows):
  counts=class_counts(rows)
  impurity=1
  for lbl in counts:
    prob_of_lbl=counts[lbl]/float(len(rows))
    impurity-=prob_of_lbl**2
  return impurity

In [ ]:
# It calculates uncertainity 
def info_gain(left,right,current_uncertainity):
  p=float(len(left))/(len(left)+len(right))
  return current_uncertainity-p*gini(left)-(1-p)*gini(right)

In [ ]:
# it is to find the best question to ask by iterating over every feature/value and calculating the information gain
def find_best_split(rows):
  best_gain=0
  best_question=None
  current_uncertainity=gini(rows)
  n_features=len(rows[0])-1
  for col in range(n_features):
    values=set([row[col] for row in rows])
    for val in values:
      question=Question(col,val)
      true_rows,false_rows=partition(rows,question)
      if len(true_rows)==0 or len(false_rows)==0:
        continue
      gain=info_gain(true_rows,false_rows,current_uncertainity)
      if gain>=best_gain:
        best_gain,best_question=gain,question
  return best_gain,best_question

In [ ]:
# It holds a dictionary of class i.e-the number of times it appears in the rows from the training data that reach this leaf
class Leaf:
  def __init__(self,rows):
    self.predictions=class_counts(rows)

In [ ]:
# In this Decision_Node asks a question
# This holds a reference to the question, and to the two child nodes
class Decision_Node:
  def __init__(self,question,true_branch,false_branch):
    self.question=question
    self.true_branch=true_branch
    self.false_branch=false_branch

In [ ]:
# Building Trees
def build_tree(rows):
  # Partitioning the data,calculating the gain and returning the question
  gain,question=find_best_split(rows)
  if gain==0:
    return Leaf(rows)
  true_rows,false_rows=partition(rows,question)
  # Recursively build the true branch
  true_branch=build_tree(true_rows)
  # Recursively build the false branch
  false_branch=build_tree(false_rows)
  return Decision_Node(question,true_branch,false_branch)

In [ ]:
# we are printing the tree
def print_tree(node,spacing=""):
  # we've reached a leaf
  if isinstance(node,Leaf):
    print(spacing + "Predict",node.predictions)
    return
    # print the question at this node
    print(spacing + str(node.question))
    # call this function recursively on the true branch
    print(spacing + '--> True:')
    print_tree(node.true_branch,spacing + " ")
    # call this function recursively on the false branch
    print(spacing + '-->False:')
    print_tree(node.false_branch,spacing + " ")

In [ ]:
def classify(row,node):
  # Base case:we've reached a Leaf
  if isinstance(node,Leaf):
    return node.predictions
  # to decide whether to follow the true-branch or the false-branch
  # compare the feature/value stored in the node, to the example we're considering
  if node.question.match(row):
    return classify(row,node.true_branch)
  else:
    return classify(row,node.false_branch)

In [ ]:
def print_leaf(counts):
  # Print the predictions at a leaf
  total=sum(counts.values())*1.0
  probs={}
  for lbl in counts.keys():
    probs[lbl]=str(int(counts[lbl]/total*100))+"%"
  return probs

In [ ]:
if __name__=='__main__':
  tree=build_tree(training_data)
  print_tree(tree)
  #Evaluate
  testing_data=[['Green',3,'Apple'],['Yellow',4,'Apple'],['Red',2,'Grape'],['Red',1,'Grape'],['Yellow',3,'Lemon']]
  for row in testing_data:
    print("Actual:%s. Predicted: %s" %
          (row[-1],print_leaf(classify(row,tree))))

Actual:Apple. Predicted: {'Mango': '100%'}
Actual:Apple. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual:Grape. Predicted: {'Grape': '100%'}
Actual:Grape. Predicted: {'Grape': '100%'}
Actual:Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}
